In [1]:
ngrok_tunnel = "https://b665-35-196-135-59.ngrok-free.app"

In [2]:
import os
os.environ["OLLAMA_HOST"] = ngrok_tunnel

In [3]:
from langchain.llms import Ollama
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import CSVLoader, PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain, RetrievalQA

In [4]:
llm = Ollama(model="llama2")

In [5]:
embeddings = OllamaEmbeddings(model="llama2")

In [6]:
dictionary = "../data/Data Dictionary.pdf"
data = "../data/data.csv"

In [7]:
dict_loader = PyPDFLoader(dictionary)
data_loader = CSVLoader(data)

In [8]:
data_docs = data_loader.load()

In [9]:
print(len(data_docs))

257063


In [10]:
os.environ["OLLAMA_HOST"] = "https://b665-35-196-135-59.ngrok-free.app"

In [17]:
from langchain_text_splitters import CharacterTextSplitter

In [16]:
# data_chroma_db = Chroma.from_documents(data_docs, embeddings, persist_directory="../data_chroma_db")
data_faiss_db = FAISS.from_documents(data_docs[0:100], embeddings)

In [18]:
data_faiss_db.save_local("../data_faiss_db")

In [12]:
dict_doc = dict_loader.load()

In [13]:
print(dict_doc[0])

page_content='Data\nDictionary\nColumn\nDescription\nclnt_id\nClient\nID\nbank_id\nBank\nID\nacc_id\nAccount\nID\ntxn_id\nTransaction\nID\ntxn_date\nTransaction\ndate\ndesc\nDescription\namt\nAmount\ncat\nCategory\nof\nthe\ntransaction\nmerchant\nMerchant\nof\nthe\ntransaction' metadata={'source': '../data/Data Dictionary.pdf', 'page': 0}


In [14]:
dict_faiss_db = FAISS.from_documents(dict_doc, embeddings)

In [15]:
dict_faiss_db.save_local("../dict_faiss_db")

In [19]:
data_retriever = data_faiss_db.as_retriever()

In [57]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [76]:
template="""
        Given these transactions: {context}
        
        for a client with clnt_id: {client_id}
        ,please answer the question: {question}
        
        Make sure you answer in a human readable sentence.
        Answer directly, don't refer to the how you answered the question.
        
        This is what each column/keyword means:
        Column ->> Description
        clnt_id ->> Client ID
        bank_id ->>  Bank ID
        acc_id ->>  Account ID
        txn_id ->> Transaction ID
        txn_date ->> Transaction date
        desc ->> Description
        amt ->> Amount
        cat ->> Category of the transaction
        merchant ->> Merchant of the transaction
        """
        
prompt = ChatPromptTemplate.from_template(template)

In [81]:
def invoke_chain(question: str, client_id=6):
    chain = (
        {
          "client_id": client_id, 
            'context': question | data_retriever, "question": question, 
         }
        | prompt
        | llm
        | StrOutputParser()
    )
    
    chain.invoke({
        "client_id": client_id,
        "query": question
    })

In [82]:
invoke_chain("How much did I spend on 01/08/2023?", client_id=6)

In [50]:
# def invoke_chain(query: str, client_id):
#     qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=data_retriever, chain_type="stuff", chain_type_kwargs={"prompt": prompt_template.format(client_id=client_id)},)
#     qa_chain({
#     'query': query,
# })

In [51]:
invoke_chain(query="How much did I spend on 01/08/2023?", client_id=6)

KeyError: 'context'

In [45]:
qa_chain({
    'client_id': '6',
    'query':'How much did I spend on 01/08/2023. My client Id is 6.',
})

ValueError: Missing some input keys: {'client_id'}